In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from PIL import Image
from IPython.core.display import HTML
from functools import partial
import cv2

from visprog.engine.utils import ProgramGenerator, ProgramInterpreter
from visprog.prompts.clevrer import create_prompt
import base64
import numpy as np
from tqdm import tqdm

In [ ]:
module_list = ["LOC", "TRACK", "GRAY", "RED", "BLUE", "GREEN", "CUBE", "SPHERE", "RUBBER", "LEFT", "TOP", "LEFTOF", "FRONTOF", "EVENT", "BEFORE", "EVAL", "RESULT"]
new_modules = ["METAL", "RIGHTOF", "CYAN"]
module_list += new_modules 
use_precomputed = True
interpreter = ProgramInterpreter(dataset='clevrer', use_precomputed=use_precomputed, module_list=module_list)


In [ ]:
prog = "OBJ0=LOC(video=VIDEO,object='object')\nOBJT=TRACK(object=OBJ0)\nPRED0=FRONTOF(object1=OBJT,var1='o0',object2=OBJT,var2='o1')\nPRED1=LEFTOF(object1=OBJT,var1='o2',object2=OBJT,var2='o1')\nPRED2=METAL(object=OBJT,var='o1')\nPRED3=RIGHTOF(object1=OBJT,var1='o0',object2=OBJT,var2='o2')\nEVENT0=EVENT(predicates=[PRED0,PRED1,PRED2,PRED3],min_duration=15)\nPRED4=LEFT(object=OBJT,var='o0')\nEVENT1=EVENT(predicates=[PRED4],min_duration=5)\nEVENT2=BEFORE(event1=EVENT0, event2=EVENT1)\nPRED5=CYAN(object=OBJT,var='o2')\nPRED6=LEFT(object=OBJT,var='o1')\nEVENT3=EVENT(predicates=[PRED5,PRED6],min_duration=1)\nEVENT4=BEFORE(event1=EVENT2, event2=EVENT3)\nANSWER0=EVAL(expr=\"'yes' if len({EVENT4}) else 'no'\")\nFINAL_RESULT=RESULT(var=ANSWER0)"

In [ ]:
output = []
for vid in tqdm([2807]):
    video = []
    if not use_precomputed:
        cap = cv2.VideoCapture(
            os.path.join(
                '/gscratch/balazinska/enhaoz/VOCAL-UDF/data',
                'clevrer',
                f'video_{str(vid//1000*1000).zfill(5)}-{str((vid//1000+1)*1000).zfill(5)}',
                f"video_{str(vid).zfill(5)}.mp4"
            )
        )
        fid = 0
        while True:
            ret, frame = cap.read()  # Read the next frame from the video
            if not ret:
                break  # Break the loop if there are no more frames

            # Convert the frame from BGR (OpenCV format) to RGB (PIL format)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Convert the frame to a PIL Image and append to the list
            image = Image.fromarray(frame)
            image.thumbnail((640,640),Image.Resampling.LANCZOS)
            video.append(image)
            fid += 1
    init_state = dict(
        VIDEO=video,
        vid=vid
    )
    result, prog_state = interpreter.execute(prog,init_state,inspect=False)
    if result == 'yes':
        output.append(vid)
print(output)